import pandas as pd

In [1]:
import pandas as pd

In [2]:
tripadv_review = pd.read_csv(r'C:\Users\Dell\Desktop\Attractions\The Venetian Column.csv', error_bad_lines=False);

In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [5]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [7]:
processed_reviews = tripadv_review['r_text'].map(preprocess)
processed_reviews

0     [venetian, column, located, main, square, nort...
1     [column, great, staggering, adjective, immedia...
2       [woooow, time, love, place, taxi, stop, colomn]
3     [marked, free, near, courts, walk, past, follo...
4     [column, stands, town, area, brought, ruins, s...
5     [worth, having, look, cross, turkish, nicosia,...
6     [centre, ataturk, square, column, called, vene...
7     [wandered, commercial, section, lefkosa, looki...
8     [small, column, centre, square, certainly, his...
9     [column, stand, tall, class, column, monument,...
10    [worth, having, look, cross, turkish, nicosia,...
11    [centre, ataturk, square, column, called, vene...
12    [wandered, commercial, section, lefkosa, looki...
13    [small, column, centre, square, certainly, his...
14    [column, stand, tall, class, column, monument,...
15    [road, bother, look, interesting, column, spec...
16    [column, situated, turkish, half, nicosia, str...
17    [overrated, monument, handy, landmark, hel

In [8]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(processed_reviews)

# Create Corpus
texts = processed_reviews

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 4), (19, 1)]]


In [17]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7, 
                                           random_state=50,
                                           update_every=1,
                                           chunksize=50,
                                           passes=5,
                                           alpha='auto',
                                           eta='auto',
                                           per_word_topics=True,
                                           iterations = 25,
                                           eval_every = None)

In [18]:
from pprint import pprint

In [19]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.023*"pigeons" + 0.023*"gather" + 0.023*"northern" + 0.023*"tower" + '
  '0.023*"nicosia" + 0.023*"number" + 0.023*"proud" + 0.023*"standing" + '
  '0.023*"different" + 0.023*"tall"'),
 (1,
  '0.059*"square" + 0.052*"column" + 0.030*"ataturk" + 0.023*"nicosia" + '
  '0.021*"british" + 0.017*"cyprus" + 0.017*"took" + 0.016*"lion" + '
  '0.014*"called" + 0.012*"worth"'),
 (2,
  '0.056*"column" + 0.035*"nicosia" + 0.031*"square" + 0.026*"turkish" + '
  '0.021*"city" + 0.021*"place" + 0.016*"area" + 0.016*"great" + 0.016*"north" '
  '+ 0.016*"monument"'),
 (3,
  '0.055*"column" + 0.029*"square" + 0.028*"bankasi" + 0.028*"wandered" + '
  '0.028*"worth" + 0.028*"centre" + 0.028*"grand" + 0.028*"looking" + '
  '0.028*"section" + 0.028*"lefkosa"'),
 (4,
  '0.006*"monument" + 0.006*"square" + 0.005*"green" + 0.005*"striking" + '
  '0.005*"trees" + 0.005*"nicosia" + 0.005*"column" + 0.005*"situated" + '
  '0.005*"clean" + 0.005*"visit"'),
 (5,
  '0.053*"column" + 0.033*"worth" + 0.023*"

In [20]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.016669 -0.111158       1        1  32.614117
1      0.143588 -0.025038       2        1  31.097054
6      0.022997  0.137623       3        1  13.681889
5     -0.111557  0.010363       4        1  11.813513
3     -0.021666  0.031877       5        1   7.910954
0     -0.013647 -0.027445       6        1   2.532866
4     -0.003045 -0.016223       7        1   0.349604, topic_info=        Term       Freq      Total Category  logprob  loglift
19  venetian   8.000000   8.000000  Default  30.0000  30.0000
13   nicosia   9.000000   9.000000  Default  29.0000  29.0000
5     column  26.000000  26.000000  Default  28.0000  28.0000
66     worth   5.000000   5.000000  Default  27.0000  27.0000
70    centre   4.000000   4.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
65   turkish   0.007661   6.122698   Topic7  -5.3892  -1.0275
0    ataturk   0.007714   8.226661   Topic7  -5.3824  -1.3159
19  venetian   0.007611   8.006523   Topic7  -5.3958  -1.3023
37     place   0.007394   4.371825   Topic7  -5.4248  -0.7262
3       city   0.007333   3.727810   Topic7  -5.4330  -0.5750

[363 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
190       2  0.747870   amazing
227       1  0.750885   ancient
228       1  0.750886  animated
87        5  0.650479    appeal
55        1  0.658696      area
...     ...       ...       ...
66        2  0.344880     worth
66        4  0.344880     worth
66        5  0.172440     worth
164       1  0.517469    wouldn
164       4  0.517469    wouldn

[229 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 7, 6, 4, 1, 5])